In [1]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

True

In [2]:
%pip install --pre -U langchain


[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# Test if we have the latest LangChain functionality
# Even though version shows 1.0.0a13, we actually have 1.0.0a14 installed

try:
    from langchain.chat_models import init_chat_model
    print("✅ init_chat_model is available (1.0.0a14 feature)")
except ImportError as e:
    print(f"❌ init_chat_model not available: {e}")

# Check if we have other 1.0.0a14 features
try:
    import langchain
    # Check for new modules that might be in 1.0.0a14
    if hasattr(langchain, 'chat_models'):
        print("✅ langchain.chat_models module is available")
    else:
        print("❌ langchain.chat_models module not found")
except Exception as e:
    print(f"Error checking features: {e}")

print(f"\nNote: Version string shows {langchain.__version__} but package is actually 1.0.0a14")
print("This is a known packaging issue with the pre-release version.")


In [ ]:
# Check LangChain version after kernel restart
import langchain
print(f"LangChain version: {langchain.__version__}")

# Verify we're using the updated version
if langchain.__version__ == "1.0.0a14":
    print("✅ Successfully updated to latest version!")
else:
    print(f"⚠️  Still on version {langchain.__version__}, expected 1.0.0a14")


In [ ]:
# Check which Python environment the notebook is using
import sys
print(f"Python executable: {sys.executable}")
print(f"Python version: {sys.version}")

# Check if we're in a virtual environment
import os
venv_path = os.environ.get('VIRTUAL_ENV')
if venv_path:
    print(f"Virtual environment: {venv_path}")
else:
    print("No virtual environment detected")


In [3]:

import langchain
print(f"LangChain version: {langchain.__version__}")

LangChain version: 1.0.0a13


In [ ]:
from langchain.agents import create_agent
from langchain.chat_models import init_chat_model
from langchain.messages import ToolMessage
from langchain_core.tools import tool
from pydantic import BaseModel, Field
from langchain_core.messages import AIMessage, SystemMessage, HumanMessage
from langsmith import traceable
import base64
import httpx

# create_agent

In [ ]:

def get_weather(city: str) -> str:
    """Get weather for a given city."""
    return f"It's always sunny in {city}!"

agent = create_agent(
    model="anthropic:claude-3-7-sonnet-latest",
    tools=[get_weather],
    system_prompt="You are a helpful assistant",
)

# Run the agent
agent.invoke(
    {"messages": [{"role": "user", "content": "what is the weather in sf"}]}
)

## Additional fields in inputs renders weird 
- Doesnt have the "Additional fields" callout like the output does
- Shows up BEFORE the list of messages (should be after)

# init_chat_model

In [ ]:
llm = init_chat_model("openai:gpt-4.1")
llm.invoke("Why do parrots talk?")

model = init_chat_model("openai:gpt-4.1")



In [ ]:
inputs = {
  "messages": [
    {
      "id": "msg_123",
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": "What breed is this dog?"
        },
        {
          "type": "image",
          "source_type": "url",
          "url": "https://fastly.picsum.photos/id/237/200/300.jpg?hmac=TmmQSbShHz9CdQm0NkEjx1Dyh_Y984R9LpNrpvH2D_U",
          "mime_type": "image/jpeg"
        }
      ]
    }
  ]
}

llm.invoke(inputs["messages"])


# model_with_tools

In [ ]:
@tool
def get_weather(location: str) -> str:
    """Get the weather at a location."""
    return f"It's sunny in {location}."


model_with_tools = llm.bind_tools([get_weather])

response = model_with_tools.invoke("What's the weather like in Boston?")

# structured outputs

In [ ]:
class Movie(BaseModel):
    """A movie with details."""
    title: str = Field(..., description="The title of the movie")
    year: int = Field(..., description="The year the movie was released")
    director: str = Field(..., description="The director of the movie")
    rating: float = Field(..., description="The movie's rating out of 10")

model_with_structure = llm.with_structured_output(Movie, include_raw=True)
response = model_with_structure.invoke("Provide details about the movie Inception")
print(response)  # Movie(title="Inception", year=2010, director="Christopher Nolan", rating=8.8)

# tools and tool messages

In [ ]:

# After a model makes a tool call
ai_message = AIMessage(
    content=[],
    tool_calls=[{
        "name": "get_weather",
        "args": {"location": "San Francisco"},
        "id": "call_123"
    }]
)

# Execute tool and create result message
weather_result = "Sunny, 72°F"
tool_message = ToolMessage(
    content=weather_result,
    tool_call_id="call_123"  # Must match the call ID
)

# Continue conversation
messages = [
    HumanMessage("What's the weather in San Francisco?"),
    ai_message,  # Model's tool call
    tool_message,  # Tool execution result
]
response = model.invoke(messages)  # Model processes the result

# Content Blocks 

In [ ]:
from langchain_core.messages import HumanMessage

# String content
messages = [HumanMessage(content_blocks=[
    {"type": "text", "text": "Hello, how are you?"},
    {"type": "image", "url": "https://fastly.picsum.photos/id/237/200/300.jpg?hmac=TmmQSbShHz9CdQm0NkEjx1Dyh_Y984R9LpNrpvH2D_U"},
])] 

llm.invoke(messages)

In [ ]:


inputs = [HumanMessage(content_blocks=[
    {"type": "text", "text": "Hello, how are you?"},
    {"type": "image", "url": "https://fastly.picsum.photos/id/237/200/300.jpg?hmac=TmmQSbShHz9CdQm0NkEjx1Dyh_Y984R9LpNrpvH2D_U"},
])] 


output_msg = AIMessage(
    content=[
        {"type": "thinking", "thinking": "...", "signature": "WaUjzkyp..."},
        {"type": "text", "text": "..."},
    ],
    response_metadata={"model_provider": "anthropic"}
)


outputs = output_msg.content_blocks


@traceable(run_type="llm")
def llm_raw(inputs):
    return outputs


llm_raw(inputs)




## Content blocks rendering

I think there's a gap in how we render content blocks: https://smith.langchain.com/o/ebbaf2eb-769b-4505-aca2-d11de10372a4/projects/p/5b35f59d-3a34-49d1-b6bb-b31d3cf45367?columnVisibilityModel=%7B%2222select%22%3Atrue%2C%22id%22%3Atrue%2C%22status%22%3Atrue%2C%22name%22%3Atrue%2C%22inputs%22%3Atrue%2C%22outputs%22%3Atrue%2C%22start_time%22%3Atrue%2C%22latency%22%3Atrue%2C%22in_dataset%22%3Atrue%2C%22last_queued_at%22%3Atrue%2C%22total_tokens%22%3Atrue%2C%22total_cost%22%3Atrue%2C%22first_token_time%22%3Atrue%2C%22tags%22%3Atrue%2C%22metadata%22%3Atrue%2C%22feedback_stats%22%3Atrue%2C%22reference_example_id%22%3Atrue%2C%22actions%22%3Atrue%7D&timeModel=%7B%22duration%22%3A%227d%22%7D&peek=bdb7da3d-14ef-4257-aa8d-35ab9becc8a7&peeked_trace=bdb7da3d-14ef-4257-aa8d-35ab9becc8a7

In [ ]:
from langchain.chat_models import init_chat_model

model = init_chat_model("openai:gpt-4o", output_version="v1")
resp = model.invoke("hello")
print (resp.content_blocks)


# Multi-modal

In [ ]:
#images

image_url = "https://fastly.picsum.photos/id/237/200/300.jpg?hmac=TmmQSbShHz9CdQm0NkEjx1Dyh_Y984R9LpNrpvH2D_U"
image_base64 = base64.standard_b64encode(httpx.get(image_url).content).decode("utf-8")


inputs = {
    "role": "user",
    "content": [
        {"type": "text", "text": "Describe the content of this image."},
        {"type": "image", "url": "https://fastly.picsum.photos/id/237/200/300.jpg?hmac=TmmQSbShHz9CdQm0NkEjx1Dyh_Y984R9LpNrpvH2D_U"},
    ]
}

# From base64 data
output = {
    "role": "user",
    "content": [
        {"type": "text", "text": "Describe the content of this image."},
        {
            "type": "image",
            "base64": image_base64,
            "mime_type": "image/jpeg",
        },
    ]
} 

@traceable(run_type="llm")
def llm_raw(inputs):
    return output


llm_raw(inputs)


## Gap in image rendering 

We don't render images
https://smith.langchain.com/o/ebbaf2eb-769b-4505-aca2-d11de10372a4/projects/p/5b35f59d-3a34-49d1-b6bb-b31d3cf45367?columnVisibilityModel=%7B%2222select%22%3Atrue%2C%22id%22%3Atrue%2C%22status%22%3Atrue%2C%22name%22%3Atrue%2C%22inputs%22%3Atrue%2C%22outputs%22%3Atrue%2C%22start_time%22%3Atrue%2C%22latency%22%3Atrue%2C%22in_dataset%22%3Atrue%2C%22last_queued_at%22%3Atrue%2C%22total_tokens%22%3Atrue%2C%22total_cost%22%3Atrue%2C%22first_token_time%22%3Atrue%2C%22tags%22%3Atrue%2C%22metadata%22%3Atrue%2C%22feedback_stats%22%3Atrue%2C%22reference_example_id%22%3Atrue%2C%22actions%22%3Atrue%7D&timeModel=%7B%22duration%22%3A%227d%22%7D&peek=7c87a2bd-fa0e-4aa0-b8b3-2719852a8757&peeked_trace=7c87a2bd-fa0e-4aa0-b8b3-2719852a8757

In [ ]:
#pdf

pdf_url = "https://pdfobject.com/pdf/sample.pdf"
pdf_base64 = base64.standard_b64encode(httpx.get(pdf_url).content).decode("utf-8")


inputs = {
    "role": "user",
    "content": [
        {"type": "text", "text": "Describe the content of this document."},
        {"type": "file", "url": "https://pdfobject.com/pdf/sample.pdf"},
    ]
}

# From base64 data
output = {
    "role": "user",
    "content": [
        {"type": "text", "text": "Describe the content of this document."},
        {
            "type": "file",
            "base64": pdf_base64,
            "mime_type": "application/pdf",
        },
    ]
} 



@traceable(run_type="llm")
def llm_raw(inputs):
    return output


llm_raw(inputs)

## Gap in redering PDFs

Same issue with PDFs

https://smith.langchain.com/o/ebbaf2eb-769b-4505-aca2-d11de10372a4/projects/p/5b35f59d-3a34-49d1-b6bb-b31d3cf45367?columnVisibilityModel=%7B%2222select%22%3Atrue%2C%22id%22%3Atrue%2C%22status%22%3Atrue%2C%22name%22%3Atrue%2C%22inputs%22%3Atrue%2C%22outputs%22%3Atrue%2C%22start_time%22%3Atrue%2C%22latency%22%3Atrue%2C%22in_dataset%22%3Atrue%2C%22last_queued_at%22%3Atrue%2C%22total_tokens%22%3Atrue%2C%22total_cost%22%3Atrue%2C%22first_token_time%22%3Atrue%2C%22tags%22%3Atrue%2C%22metadata%22%3Atrue%2C%22feedback_stats%22%3Atrue%2C%22reference_example_id%22%3Atrue%2C%22actions%22%3Atrue%7D&timeModel=%7B%22duration%22%3A%227d%22%7D&peek=1955e5c3-b539-4029-8321-25bb25a344b1&peeked_trace=1955e5c3-b539-4029-8321-25bb25a344b1

Not going to try audio, video but likely the same 

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.agents import ToolNode, create_agent
import random

@tool
def fetch_user_data(user_id: str) -> str:
    """Fetch user data from database."""
    if random.random() > 0.7:
        raise ConnectionError("Database connection timeout")
    return f"User {user_id}: John Doe, john@example.com, Active"

@tool
def process_transaction(amount: float, user_id: str) -> str:
    """Process a financial transaction."""
    if amount > 10000:
        raise ValueError(f"Amount {amount} exceeds maximum limit of 10000")
    return f"Processed ${amount} for user {user_id}"

def handle_errors(e: Exception) -> str:
    if isinstance(e, ConnectionError):
        return "The database is currently overloaded, but it is safe to retry. Please try again with the same parameters."
    elif isinstance(e, ValueError):
        return f"Error: {e}. Try to process the transaction in smaller amounts."
    return f"Error: {e}. Please try again."

tool_node = ToolNode(
    tools=[fetch_user_data, process_transaction],
    handle_tool_errors=handle_errors
)

agent = create_agent(
    model=ChatOpenAI(model="gpt-4o"),
    tools=tool_node,
    prompt="You are a financial assistant."
)

agent.invoke({
    "messages": [{"role": "user", "content": "Process a payment of 15000 dollars for user123. Generate a receipt email and address it to the user."}]
})

In [ ]:

@tool
def search(query: str) -> str:
    """Search for information."""
    return f"Results for: {query}"

@tool
def calculate(expression: str) -> str:
    """Perform mathematical calculations."""
    return 99

agent = create_agent(model, tools=[search, calculate])


agent.invoke({
    "messages": [SystemMessage(content="always prefer to use a tool if it is relevant to the user's query"),
    HumanMessage(content="what is 2 + 2?")]
})

In [ ]:
# human_in_the_loop_demo.py

import json
from typing import Any, Literal, Protocol

# --- LangChain / LangGraph imports ---
from langchain_core.tools import tool
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage, ToolCall, ToolMessage
from langchain.agents import create_agent
from langchain.agents.middleware.types import AgentMiddleware, AgentState
from langgraph.types import interrupt, Command
from langgraph.runtime import Runtime
from langgraph.checkpoint.memory import InMemorySaver
from typing_extensions import NotRequired, TypedDict


# =============================================================================
# 1) Human-in-the-Loop middleware (your implementation)
# =============================================================================

class Action(TypedDict):
    """Represents an action with a name and arguments."""
    name: str
    arguments: dict[str, Any]


class ActionRequest(TypedDict):
    """Represents an action request with a name, arguments, and description."""
    name: str
    arguments: dict[str, Any]
    description: NotRequired[str]


DecisionType = Literal["approve", "edit", "reject"]


class ReviewConfig(TypedDict):
    """Policy for reviewing a HITL request."""
    action_name: str
    allowed_decisions: list[DecisionType]
    arguments_schema: NotRequired[dict[str, Any]]


class HITLRequest(TypedDict):
    """Request for human feedback on a sequence of actions requested by a model."""
    action_requests: list[ActionRequest]
    review_configs: list[ReviewConfig]


class ApproveDecision(TypedDict):
    type: Literal["approve"]


class EditDecision(TypedDict):
    type: Literal["edit"]
    edited_action: Action


class RejectDecision(TypedDict):
    type: Literal["reject"]
    message: NotRequired[str]


Decision = ApproveDecision | EditDecision | RejectDecision


class HITLResponse(TypedDict):
    decisions: list[Decision]


class _DescriptionFactory(Protocol):
    def __call__(self, tool_call: ToolCall, state: AgentState, runtime: Runtime) -> str: ...


class InterruptOnConfig(TypedDict):
    """Configuration for an action requiring human in the loop."""
    allowed_decisions: list[DecisionType]
    description: NotRequired[str | _DescriptionFactory]
    arguments_schema: NotRequired[dict[str, Any]]


class HumanInTheLoopMiddleware(AgentMiddleware):
    """Human in the loop middleware."""

    def __init__(
        self,
        interrupt_on: dict[str, bool | InterruptOnConfig],
        *,
        description_prefix: str = "Tool execution requires approval",
    ) -> None:
        super().__init__()
        resolved_configs: dict[str, InterruptOnConfig] = {}
        for tool_name, tool_config in interrupt_on.items():
            if isinstance(tool_config, bool):
                if tool_config is True:
                    resolved_configs[tool_name] = InterruptOnConfig(
                        allowed_decisions=["approve", "edit", "reject"]
                    )
            elif tool_config.get("allowed_decisions"):
                resolved_configs[tool_name] = tool_config
        self.interrupt_on = resolved_configs
        self.description_prefix = description_prefix

    def _create_action_and_config(
        self,
        tool_call: ToolCall,
        config: InterruptOnConfig,
        state: AgentState,
        runtime: Runtime,
    ) -> tuple[ActionRequest, ReviewConfig]:
        tool_name = tool_call["name"]
        tool_args = tool_call["args"]

        # Description can be a string or callable
        description_value = config.get("description")
        if callable(description_value):
            description = description_value(tool_call, state, runtime)
        elif description_value is not None:
            description = description_value
        else:
            description = f"{self.description_prefix}\n\nTool: {tool_name}\nArgs: {tool_args}"

        action_request = ActionRequest(
            name=tool_name,
            arguments=tool_args,
            description=description,
        )
        review_config = ReviewConfig(
            action_name=tool_name,
            allowed_decisions=config["allowed_decisions"],
        )
        return action_request, review_config

    def _process_decision(
        self,
        decision: Decision,
        tool_call: ToolCall,
        config: InterruptOnConfig,
    ) -> tuple[ToolCall | None, ToolMessage | None]:
        allowed_decisions = config["allowed_decisions"]

        if decision["type"] == "approve" and "approve" in allowed_decisions:
            return tool_call, None

        if decision["type"] == "edit" and "edit" in allowed_decisions:
            edited_action = decision["edited_action"]
            return (
                ToolCall(
                    type="tool_call",
                    name=edited_action["name"],
                    args=edited_action["arguments"],
                    id=tool_call["id"],
                ),
                None,
            )

        if decision["type"] == "reject" and "reject" in allowed_decisions:
            content = decision.get("message") or (
                f"User rejected the tool call for `{tool_call['name']}` with id {tool_call['id']}"
            )
            tool_message = ToolMessage(
                content=content,
                name=tool_call["name"],
                tool_call_id=tool_call["id"],
                status="error",
            )
            return tool_call, tool_message

        msg = (
            f"Unexpected human decision: {decision}. "
            f"Decision type '{decision.get('type')}' "
            f"is not allowed for tool '{tool_call['name']}'. "
            f"Expected one of {allowed_decisions}."
        )
        raise ValueError(msg)

    def after_model(self, state: AgentState, runtime: Runtime) -> dict[str, Any] | None:
        """Trigger interrupt flows for relevant tool calls after an AIMessage."""
        messages = state["messages"]
        if not messages:
            return None

        last_ai_msg = next((m for m in reversed(messages) if isinstance(m, AIMessage)), None)
        if not last_ai_msg or not last_ai_msg.tool_calls:
            return None
        
        # ✅ NEW: If any earlier AI message already proposed tool calls,
        # skip HITL (we only interrupt on the *first* tool-call round).
        prior_ai_tool_msgs = [
            m for m in messages
            if isinstance(m, AIMessage) and (m is not last_ai_msg) and getattr(m, "tool_calls", None)
        ]
        if prior_ai_tool_msgs:
            return None

        interrupt_tool_calls: list[ToolCall] = []
        auto_approved_tool_calls: list[ToolCall] = []

        for tool_call in last_ai_msg.tool_calls:
            (interrupt_tool_calls if tool_call["name"] in self.interrupt_on
             else auto_approved_tool_calls).append(tool_call)

        if not interrupt_tool_calls:
            return None

        revised_tool_calls: list[ToolCall] = auto_approved_tool_calls.copy()
        artificial_tool_messages: list[ToolMessage] = []

        action_requests: list[ActionRequest] = []
        review_configs: list[ReviewConfig] = []

        for tool_call in interrupt_tool_calls:
            cfg = self.interrupt_on[tool_call["name"]]
            action_request, review_config = self._create_action_and_config(
                tool_call, cfg, state, runtime
            )
            action_requests.append(action_request)
            review_configs.append(review_config)

        hitl_request = HITLRequest(
            action_requests=action_requests,
            review_configs=review_configs,
        )

        resp = interrupt(hitl_request)
    
        # Some runtimes return a single response, others wrap it in a list.
        if isinstance(resp, list):
            if len(resp) != 1:
                raise ValueError(f"Expected exactly 1 HITLResponse, got {len(resp)}")
            resp = resp[0]
        decisions = resp["decisions"]


        if (len(decisions)) != (len(interrupt_tool_calls)):
            raise ValueError(
                f"Number of human decisions ({len(decisions)}) does not match "
                f"number of hanging tool calls ({len(interrupt_tool_calls)})."
            )

        for i, decision in enumerate(decisions):
            tool_call = interrupt_tool_calls[i]
            cfg = self.interrupt_on[tool_call["name"]]
            revised_tool_call, tool_message = self._process_decision(decision, tool_call, cfg)
            if revised_tool_call:
                revised_tool_calls.append(revised_tool_call)
            if tool_message:
                artificial_tool_messages.append(tool_message)

        last_ai_msg.tool_calls = revised_tool_calls
        return {"messages": [last_ai_msg, *artificial_tool_messages]}


# =============================================================================
# 2) Tools
# =============================================================================

@tool
def search(query: str) -> str:
    """Search for information."""
    return f"Results for: {query}"

@tool
def calculate(expression: str) -> str:
    """Perform mathematical calculations."""
    # Intentional value to make HITL visible when you edit/approve
    # You can change to eval(expression) after testing.
    return "99"


# =============================================================================
# 3) Agent + Middleware + Checkpointer
# =============================================================================

# TODO: Replace with your real chat model, e.g.:
from langchain_openai import ChatOpenAI
model = ChatOpenAI(model="gpt-4o", temperature=0)

hitl = HumanInTheLoopMiddleware(
    interrupt_on={
        # Interrupt AFTER model proposes these tool calls
        "search": True,
        "calculate": True,
    },
    description_prefix="Pending approval:",
)

agent = create_agent(
    model,
    tools=[search, calculate],
    middleware=[hitl],
    checkpointer=memory,
)

memory = InMemorySaver() 
thread_id = "demo-thread-8"


# =============================================================================
# 4) Terminal-driven human review loop
# =============================================================================

import json
from langgraph.types import Command
from langchain_core.messages import AIMessage

def _normalize_interrupts(raw_interrupts):
    """Return a list of plain dict requests (unwraps Interrupt objects)."""
    # Make it a list
    items = raw_interrupts if isinstance(raw_interrupts, list) else [raw_interrupts]
    out = []
    for item in items:
        # Unwrap Interrupt objects (they usually carry the payload in .value)
        item = getattr(item, "value", item)
        out.append(item)
    return out

def run_with_review(user_text: str):
    result = agent.invoke(
        {
            "messages": [
                SystemMessage(content="always prefer to use a tool if it is relevant to the user's query"),
                HumanMessage(content=user_text),
            ]
        },
        config={
            "configurable": {"thread_id": thread_id},
            "checkpointer": memory,
        },
    )

    while isinstance(result, dict) and "__interrupt__" in result:
        raw_interrupts = result["__interrupt__"]
        requests = _normalize_interrupts(raw_interrupts)  # <-- key fix

        responses = []
        for req_idx, req in enumerate(requests, 1):
            # req is now a plain dict (your HITLRequest): has keys action_requests, review_configs
            actions = req.get("action_requests", [])
            print("\n=== HUMAN REVIEW REQUIRED ===")
            print(f"(interrupt #{req_idx} with {len(actions)} action(s))")
            for i, item in enumerate(actions, 1):
                name = item["name"]
                args = item.get("arguments")
                desc = item.get("description")
                print(f"[{i}] {name}  args={args}")
                if desc:
                    print(f"    desc: {desc}")

            print("\nEnter actions comma-separated for THIS interrupt (approve|edit|reject). Example: approve,edit")
            choices = [c.strip().lower() for c in input("> ").split(",")]
            if len(choices) != len(actions):
                print("Count mismatch; defaulting any missing to 'approve'.")
                choices += ["approve"] * (len(actions) - len(choices))

            decisions = []
            for choice, item in zip(choices, actions):
                if choice == "approve":
                    decisions.append({"type": "approve"})
                elif choice == "edit":
                    print(f"Enter edited tool name (or blank to keep '{item['name']}'):")
                    new_name = input("> ").strip() or item["name"]
                    print('Enter edited JSON args on one line (e.g., {"expression": "2+2"}):')
                    raw = input("> ").strip() or "{}"
                    try:
                        new_args = json.loads(raw)
                        if not isinstance(new_args, dict):
                            raise ValueError("edited arguments must be a JSON object")
                    except Exception as e:
                        print(f"Invalid JSON ({e}); using empty object.")
                        new_args = {}
                    decisions.append({
                        "type": "edit",
                        "edited_action": {"name": new_name, "arguments": new_args}
                    })
                elif choice == "reject":
                    print("Enter a short rejection message:")
                    msg = input("> ").strip() or "Rejected by human reviewer."
                    decisions.append({"type": "reject", "message": msg})
                else:
                    decisions.append({"type": "approve"})

            responses.append({"decisions": decisions})

        # Resume with one HITLResponse per interrupt request (order matters)
        result = agent.invoke(
            Command(resume=responses),
            config={
                "configurable": {"thread_id": thread_id},
                "checkpointer": memory,
            },
        )

    if isinstance(result, AIMessage):
        print("\n=== FINAL ===")
        print(result.content)
    else:
        print("\n=== FINAL (dict) ===")
        print(result)



# =============================================================================
# 5) Run
# =============================================================================

if __name__ == "__main__":
    # Example prompt that should trigger a tool call (and thus an interrupt)
    run_with_review("what is 2 + 2?")
